In [1]:
from google.colab import drive

drive.mount('/content/drive')

%cd /content/drive/MyDrive/ift6289-project/
!ls

Mounted at /content/drive
/content/drive/MyDrive/ift6289-project
'Copy of ift6289-project-facebook mbart-large-50-many-to-many-mmt-BNtoEN.ipynb'
'Copy of ift6289-project-mt5-base-ENtoBN.ipynb'
 gemma_7b_finetune_template.ipynb
 hasan-etal-2020-low
 hasan-etal-2020-low.tar.bz2
'ift6289-project-facebook mbart-large-50-many-to-many-mmt-BNtoEN.ipynb'
'ift6289-project-facebook mbart-large-50-many-to-many-mmt-ENtoBN.ipynb'
'ift6289-project-facebook nllb-200-distilled-600M.ipynb'
'ift6289-project-Helsinki-NLP opus-mt-bn-en.ipynb'
'ift6289-project-Helsinki-NLP opus-mt-en-mul.ipynb'
'ift6289-project-Helsinki-NLP opus-mt-mul-en.ipynb'
'ift6289-project-meta-llama Llama-2-7b-hf.ipynb'
 ift6289-project-mt5-base-BNtoEN.ipynb
 ift6289-project-mt5-base-ENtoBN.ipynb
 ift6289-project-umt5-base-BNtoEN.ipynb
 ift6289-project-umt5-base-ENtoBN.ipynb
 logs
 results


In [2]:
!pip install datasets transformers[sentencepiece] sacrebleu
!pip install transformers
!pip install ml_things
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 42.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 8.4 MB/s eta 0:00:00
  Created wheel for ml_things: filename=ml_things-0.0.1-py3-none-any.whl size=24393 sha256=023953209fd17303deb4

In [3]:
from transformers import set_seed, AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, MT5ForConditionalGeneration, T5Tokenizer, DataCollatorForSeq2Seq, pipeline
from datasets import Dataset, load_dataset, load_metric
import torch
import pandas as pd
from tqdm.auto import tqdm  # tqdm is great for progress bars
import sacrebleu
import numpy as np

In [4]:
set_seed(42)

dataset = load_dataset("csebuetnlp/BanglaNMT")
# print(dataset)
# print(dataset['train'][0])
# print(dataset['train'][1])
# print(dataset['train'][2])
# print(dataset['train'][0]['bn'])
# print(dataset['train'][0]['en'])

training_subset_size = 100000
dataset["train"] = dataset["train"].select(range(training_subset_size))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/2379749 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/597 [00:00<?, ? examples/s]

In [5]:
# Specify the model checkpoint
# model_checkpoint = "facebook/nllb-200-distilled-600M"
# model_checkpoint = "Helsinki-NLP/opus-mt-bn-en"
model_checkpoint = "google/mt5-base"
tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
def preprocess_function(examples):
    inputs = ["Translate Bengali to English: " + ex for ex in examples["bn"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length", return_tensors="pt")
    # Prepare the labels (English translations)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['en'], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)
print(tokenized_datasets)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/597 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['bn', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['bn', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['bn', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 597
    })
})


In [7]:
for i in range(3):  # Adjust the range as needed
    print(f"\nExample {i+1}:")
    print("Input (English):", tokenizer.decode(tokenized_datasets['train'][i]['input_ids'], skip_special_tokens=True))
    print("Label (Bengali):", tokenizer.decode(tokenized_datasets['train'][i]['labels'], skip_special_tokens=True))


Example 1:
Input (English): Translate Bengali to English: আমাকে সব কিছু জানাও
Label (Bengali): Just keep me informed.

Example 2:
Input (English): Translate Bengali to English: "এ্যামোর এন্ড সাইকি", এটা খারাপ, মনিব এটায় প্রচুর পরিমাণে চিরহরিৎ (একপ্রকার সুগন্ধী লতা) আছে আর প্রচুর পরিমাণে...
Label (Bengali): It's bad, "Amor and Psyche" is, master.

Example 3:
Input (English): Translate Bengali to English: কাছানো
Label (Bengali): Overtake


In [8]:
model = MT5ForConditionalGeneration.from_pretrained(model_checkpoint)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
# Load both BLEU and SacreBLEU metrics
bleu_metric = load_metric('bleu')
sacrebleu_metric = load_metric('sacrebleu')

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    # # Compute BLEU score
    # bleu_result = bleu_metric.compute(predictions=decoded_preds, references=[decoded_labels])
    # Compute SacreBLEU score
    sacrebleu_result = sacrebleu_metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Prepare result dictionary
    results = {
        # "bleu": bleu_result['score'],
        "sacrebleu": sacrebleu_result['score']
    }

    return results

<ipython-input-9-415e82ee0514>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric('bleu')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [11]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="sacrebleu",
    # report_to="none",
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# # Pre-training test results
baseline_test_results = trainer.predict(tokenized_datasets["test"], metric_key_prefix="test")
print("Baseline Test Metrics:", baseline_test_results.metrics)
baseline_validation_results = trainer.evaluate()
print("Baseline Validation Metrics:", baseline_validation_results)

# Fine-tune the model
print("\nFine-tuning the model...")
trainer.train()
print("Model fine-tuning complete.")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Baseline Test Metrics: {'test_loss': 0.8376097679138184, 'test_sacrebleu': 0.3951838969162758, 'test_runtime': 34.7335, 'test_samples_per_second': 28.791, 'test_steps_per_second': 1.814}


Baseline Validation Metrics: {'eval_loss': 0.8432939648628235, 'eval_sacrebleu': 0.32046932849816223, 'eval_runtime': 20.9686, 'eval_samples_per_second': 28.471, 'eval_steps_per_second': 1.812}

Fine-tuning the model...


Epoch,Training Loss,Validation Loss,Sacrebleu
1,0.455800,0.365500,10.885278
2,0.359800,0.327793,13.189965
3,0.314800,0.316972,14.227716


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


Model fine-tuning complete.


In [12]:
# Evaluate the model
generation_kwargs = {
    "max_length": 258,
    "num_beams": 4,  # Example, adjust based on your needs
}
validation_results = trainer.evaluate(generation_kwargs=generation_kwargs)
print("Final validation Metrics", validation_results)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Final validation Metrics {'eval_loss': 0.31697237491607666, 'eval_sacrebleu': 14.227716083416283, 'eval_runtime': 21.3116, 'eval_samples_per_second': 28.013, 'eval_steps_per_second': 1.783, 'epoch': 3.0}


In [13]:
# Calculate scores on the test set
final_test_results = trainer.predict(tokenized_datasets["test"], metric_key_prefix="test", generation_kwargs=generation_kwargs)
print("Final Test Metrics:", final_test_results.metrics)

Final Test Metrics: {'test_loss': 0.32059580087661743, 'test_sacrebleu': 14.376077927173979, 'test_runtime': 35.3595, 'test_samples_per_second': 28.281, 'test_steps_per_second': 1.782}


In [14]:
# Decode the predictions and labels
decoded_predictions = tokenizer.batch_decode(final_test_results.predictions, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(final_test_results.label_ids, skip_special_tokens=True)

# Print a sample of the predictions and labels
print("Sample Predictions and Labels:")
for i in range(10):  # Adjust the range to print more or fewer examples
    print(f"\nPrediction {i+1}: {decoded_predictions[i]}")
    print(f"Label {i+1}: {decoded_labels[i]}")

Sample Predictions and Labels:

Prediction 1: The airport was opened by Bangladesh High Commissioner Saiida Munna Tasnim and senior
Label 1: Bangladesh High Commissioner to the United Kingdom Saida Muna Tasneen and senior officials of Bangladesh Mission in London saw him off at the airport.

Prediction 2: The terrorists have no race, religion, and country.
Label 2: The terrorists have no race, religion and countries.

Prediction 3: Derek Kabala, professor of the University of Cardiff, wrote a article titled
Label 3: Derek Cabera, professor at Cornell University, wrote an essay called, "The dark sides of categorical thinking."

Prediction 4: He said that the public could buy rice at 45 rupees, sugar at 50
Label 4: He said that people can buy onion at Tk 45 per kg, sugar at Tk 50 kg, lentil at Tk 50 kg and Soybean oil at Tk 85 per litre.

Prediction 5: The two units of Sripur Fire Station tried to fire for about two hours.
Label 5: Locals and firemen of two units of Sreepur Fire Station 

# Test on SUPara-benchmark

In [15]:
supara_source_test_path = f"hasan-etal-2020-low/data/SUPara-benchmark/suparatest2018/suparatest2018_bn.txt"
supara_target_test_path = f"hasan-etal-2020-low/data/SUPara-benchmark/suparatest2018/suparatest2018_en.txt"

with open(supara_source_test_path, "r", encoding="utf-8") as f:
    supara_source_data = f.readlines()

with open(supara_target_test_path, "r", encoding="utf-8") as f:
    supara_target_data = f.readlines()

print(supara_source_data[0])
print(supara_target_data[0])

সে লাল গোলাপ পছন্দ করে।

She likes red roses.



In [16]:
# def preprocess_function_supara(examples):
#     inputs = ["Translate Bengali to English: " + ex for ex in examples["bn"]]
#     model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length", return_tensors="pt")
#     # Prepare the labels (English translations)
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(examples['en'], max_length=128, truncation=True, padding="max_length")
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

supara_dataset = Dataset.from_dict({"bn": supara_source_data, "en": supara_target_data})
tokenized_supara_dataset = supara_dataset.map(preprocess_function, batched=True)
print(tokenized_supara_dataset)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset({
    features: ['bn', 'en', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 500
})


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [17]:
# Perform inference with the fine-tuned model
final_test_results_supara = trainer.predict(tokenized_supara_dataset, metric_key_prefix="test")
print("Final test metrics SUPara:", final_test_results_supara.metrics)
print("Final test results SUPara:", final_test_results_supara)

Final test metrics SUPara: {'test_loss': 0.3996988534927368, 'test_sacrebleu': 10.539675987117363, 'test_runtime': 17.0835, 'test_samples_per_second': 29.268, 'test_steps_per_second': 1.873}
Final test results SUPara: PredictionOutput(predictions=array([[    0,  1669,  1469, ...,     0,     0,     0],
       [    0,  1385, 34589, ...,     0,     0,     0],
       [    0,  1306,  4657, ...,     0,     0,     0],
       ...,
       [    0,   259, 39983, ...,   259, 96953,   305],
       [    0,   336,   728, ...,   259,  2015,   261],
       [    0,   259, 47482, ...,   287,   276,   418]]), label_ids=array([[ 4630,  1469,   263, ...,     0,     0,     0],
       [ 1385,   259,  8142, ...,     0,     0,     0],
       [ 1306,  4657,   281, ...,     0,     0,     0],
       ...,
       [36874,   259, 49886, ...,     0,     0,     0],
       [  486, 42893,   715, ...,     0,     0,     0],
       [  259, 47482,   259, ...,     0,     0,     0]]), metrics={'test_loss': 0.3996988534927368, '

In [18]:
# Decode the predicted sequences
decoded_predictions_supara = tokenizer.batch_decode(final_test_results_supara.predictions, skip_special_tokens=True)
labels_supara = np.where(final_test_results_supara.label_ids != -100, final_test_results_supara.label_ids, tokenizer.pad_token_id)
decoded_labels_supara = tokenizer.batch_decode(labels_supara, skip_special_tokens=True)

# Print a sample of the predictions and labels
print("Sample Predictions and Labels:")
for i in range(10):  # Adjust the range to print more or fewer examples
    print(f"\nPrediction {i+1}: {decoded_predictions_supara[i]}")
    print(f"Label {i+1}: {decoded_labels_supara[i]}")

Sample Predictions and Labels:

Prediction 1: He likes red flowers.
Label 1: She likes red roses.

Prediction 2: It strengthens our body.
Label 2: It makes our body firm.

Prediction 3: We live in Bangladesh.
Label 3: We live in Bangladesh.

Prediction 4: Human life is very short.
Label 4: Human life is very short.

Prediction 5: The traffic police have many responsibilities.
Label 5: Traffic police have heavy responsibilities.

Prediction 6: Cricket is my favorite sport.
Label 6: Cricket is my favourite game.

Prediction 7: Citizenship should be created.
Label 7: Mass awareness should be created.

Prediction 8: There may be some problems.
Label 8: There may be some difficulties.

Prediction 9: What is his name?
Label 9: What is her name?

Prediction 10: Our college is very old.
Label 10: Our college is very ancient.
